**“数字人体”视觉挑战赛-宫颈癌风险智能诊断**

**注：此note应与OneNote和Server上的jupyter notebook一起使用**

# 整体思路
1. 2019年10月23日：
先用CNN加Apooling的方法进行二分类，然后遍历全图，找到关键部分。

Q：可以借鉴之前的比赛，将问题区域掩盖？

A：不可以。官方有话：“特别注明：阳性图片ROI区域之外不保证没有异常鳞状上皮细胞”


# 数据准备
服务器上下载test数据集、neg数据集，本地以及下载安装pos样本数据集
避免影响[wget限速下载方法](https://www.cnblogs.com/xzlive/p/9584500.html)
> 限速下载，只需要添加–limit-rate=300k合理参数即可
wget -c --limit-rate=300k http://mirrors.163.com/ubuntu-releases/9.10/ubuntu-9.10-desktop-amd64.iso 



## 正样本统计
![](./Tian_20191022/w&h_percent.png)
可以看出80%以上的样本，宽高是小于200的，这个宽度不宜选的太大，不然小样本不容易找到他的bounding box

无论如何先把模型搭建好，具体的参数都是可以修改的！

## 样本增强
### 正样本
经过分析，采用crop的方式增强，因为conv_size的宽高是确定的，因此只需要确定满足条件的(x,y)点即可。
![](./Tian_20191022/iou.jpg)
而通过分析可知，只要确定左上角和右下角的坐标就可以确定满足条件的区域

具体可以分为以下三种情况：
1. region远小于conv_size，此时IOU最大也不满足，此时需要全部包含region
1. 最大IOU满足条件，需要找到左上和右下满足条件的IOU临界值，在临界值内画区域
1. region远大于conv_size，此时也不应过多的包含region外的区域，就在region内部找实际区域

1. (x-(conv_s - w),y - (conv_s - h),x,y
1. 假设w_hat是我们要求的共有面积的宽，临界值一定出现在宽高相等的地方，因此$h = w = w_{hat}$
且有：$$w_{hat}^2 = (conv_s^2 + regionarea - w_{hat}^2) * rate_{iou}$$
$$w_{hat}^2 = \frac{rate_{iou}}{1 + rate_{iou}} \times  (conv_s^2 + regionarea)$$
$$w_{hat} = \sqrt{\frac{rate_{iou}}{1 + rate_{iou}} \times  (conv_s^2 + regionarea)}$$
因此，crop坐标为：
(x-(conv_s - w_{hat}),y - (conv_s - w_{hat}),...

2019年10月23日：
不行太复杂了，还是随机crop先，目前方案是确保conv_size框的中心点在region区域内即可

### 负样本

S：了解下有没有不解压zip并读取的方法，好像并没有[参考](http://localhost:8888/notebooks/python%20learn/sys%26os%26zip.ipynb)

**1. 负样本数量**

正样本总数大概160000个，第一阶段扩充之后应该只会有64W个，所以对于300张负样本，每张随机采样2200张图片。这样比较better

**2. 负样本方法**

训练的时候直接从数据库离读吧，写到硬盘里也没啥意思

# 模型搭建
2019年10月24日：先用resnet试试

**框架**

用哪个框架呢，好像现在都有现成的预训练模型了，用`fine-tuning`改进一下试试用先

就mxnet吧，毕竟比较系统的学过